 <table><tr><td><img src="images/dbmi_logo.png" width="75" height="73" alt="Pitt Biomedical Informatics logo"></td><td><img src="images/pitt_logo.png" width="75" height="75" alt="University of Pittsburgh logo"></td></tr></table>
 
 
 # Social Media and Data Science - Part 5
 
 
Data science modules developed by the University of Pittsburgh Biomedical Informatics Training Program with the support of the National Library of Medicine data science supplement to the University of Pittsburgh (Grant # T15LM007059-30S1). 

Developed by Harry Hochheiser, harryh@pitt.edu. All errors are my responsibility.

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.


### Goal: Use social media posts to explore the appplication of text and natural language processing to see what might be learned from online interactions.

Specifically, we will retrieve, annotate, process, and interpret Twitter data on health-related issues such as smoking.

--- 
References:
* [Mining Twitter Data with Python (Part 1: Collecting data)](https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/)
* The [Tweepy Python API for Twitter](http://www.tweepy.org/)

---

In [1]:
%matplotlib inline

import operator
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime
from spacy.symbols import ORTH, LEMMA, POS

# 5.0 Introduction

This final part of our journey through social media data retrieval, annotation, natural langauge processing, and classififcation will challenge you to apply these techniques to a new problem. Specifically, you will create, annotate, and process a new data set.

# 5.0.1 Setup

As before, we start with the Tweets class and the configuration for our Twitter API connection.  We may not need this, but we'll load it in any case.

In [2]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.tweets={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        searchTime=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",tweet_mode='extended',count=corpus_size)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt,searchTime,term)
            time.sleep(30)
                
    def addTweet(self,tweet,searchTime,term="",count=0):
        id = tweet['id_str']
        if id not in self.tweets.keys():
            self.tweets[id]={}
            self.tweets[id]['tweet']=tweet
            self.tweets[id]['count']=0
            self.tweets[id]['searchTime']=searchTime
            self.tweets[id]['searchTerm']=term
        self.tweets[id]['count'] = self.tweets[id]['count'] +1
        
    def combineTweets(self,other):
        for otherid in other.getIds():
            tweet = other.getTweet(otherid)
            searchTerm = other.getSearchTerm(otherid)
            searchTime = other.getSearchTime(otherid)
            self.addTweet(tweet,searchTime,searchTerm)
        
    def getTweet(self,id):
        if id in self.tweets:
            return self.tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.tweets[id]['count']
    
    def countTweets(self):
        return len(self.tweets)
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        for t,entry in self.tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.tweets.keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.tweets)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.tweets=incontents
        
    def getSearchTerm(self,id):
        return self.tweets[id]['searchTerm']
    
    def getSearchTime(self,id):
        return self.tweets[id]['searchTime']
    
    def getText(self,id):
        tweet = self.getTweet(id)
        text=tweet['full_text']
        if 'retweeted_status'in tweet:
            original = tweet['retweeted_status']
            text=original['full_text']
        return text
                
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
   
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
 
    def getCodes(self,id):
        tweet=self.getTweet(id)
        if 'codes' in tweet:
            return tweet['codes']
        else:
            return None
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        for id in self.tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

Put the values of your keys into these variables

In [3]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

In [4]:
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

We will also load some routines that we defined in [Part 3](SocialMedia - Part 3.ipynb):
    
1. Our routine for creating a customized NLP pipeline
2. Our routine for including tokens
3. The `filterTweetTokens` routine defined in an exercise (Without the inclusion of named entities. It will be easier to leave them out for now).

In [5]:
def getTwitterNLP():
    nlp = spacy.load('en')
    
    for word in nlp.Defaults.stop_words:
        lex = nlp.vocab[word]
        lex.is_stop = True
    
    special_case = [{ORTH: u'e-cigarette', LEMMA: u'e-cigarette', POS: u'NOUN'}]
    nlp.tokenizer.add_special_case(u'e-cigarette', special_case)
    nlp.tokenizer.add_special_case(u'E-cigarette', special_case)
    vape_case = [{ORTH: u'vape',LEMMA:u'vape',POS: u'NOUN'}]
    
    vape_spellings =[u'vap',u'vape',u'vaping',u'vapor',u'Vap',u'Vape',u'Vapor',u'Vapour']
    for v in vape_spellings:
        nlp.tokenizer.add_special_case(v, vape_case)
    def hashtag_pipe(doc):
        merged_hashtag = True
        while merged_hashtag == True:
            merged_hashtag = False
            for token_index,token in enumerate(doc):
                if token.text == '#':
                    try:
                        nbor = token.nbor()
                        start_index = token.idx
                        end_index = start_index + len(token.nbor().text) + 1
                        if doc.merge(start_index, end_index) is not None:
                            merged_hashtag = True
                            break
                    except:
                        pass
        return doc
    nlp.add_pipe(hashtag_pipe,first=True)
    return nlp

def includeToken(tok):
    val =False
    if tok.is_stop == False:
        if tok.is_alpha == True: 
            if tok.text =='RT':
                val = False
            elif tok.pos_=='NOUN' or tok.pos_=='PROPN' or tok.pos_=='VERB':
                val = True
        elif tok.text[0]=='#' or tok.text[0]=='@':
            val = True
    if val== True:
        stripped =tok.lemma_.lower().strip()
        if len(stripped) ==0:
            val = False
        else:
            val = stripped
    return val

def filterTweetTokens(tokens):
    filtered=[]
    for t in tokens:
        inc = includeToken(t)
        if inc != False:
            filtered.append(inc)
    return filtered

Finally, we will include some additional modules from Scikit-Learn:

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
import string
import re

Now, we're ready to go along for an exercise

Identifying the source of social media comments might be an important step in the process of interpreting a large corpus. Continuing with our example of smoking and vaping, it might be interesting to compare tweets from users - people who are talking about their own personal use  to those who might be either promoting vaping  (manufacturers, sponsors, etc.) or warning about dangers of vaping (physicians, researchers, public health agencies, etc.).

A team of researchers at RTI International tackled this problem in a 2018 paper [Classification of Twitter Users Who Tweet About E-Cigarettes](http://publichealth.jmir.org/2017/3/e63/) by Annice Kim and colleagues collected tweets and attributed them to individuals, enthusiasts, "informed agencies (news media or health community), marketers, or spammers. 

Your goal here is to collect a small data set and to attempt a smaller version of this challenge. Specifically, we will try to collect preliminary data for a classifier capable of identifing tweets from users of e-cigarettes vs. others.  Using any of the code found in Parts 1-4, complete these steps:

1. Run some searches for tweets like 'e-cig', 'e-cigarette', 'vape' and 'vaping'. Collect a corpus of 200-300  or more tweets. You might want to save each of these result sets in files.

2. Combine these tweets into one large collection using the 'Tweet' class listed above. Save the results in a file 

3. Annotate 50 of these tweets as pertaining to either 'individual' or 'non-individual'. Be sure that you do at least a few of the tweets from each of the original sets. One way to do this might be to randomize the tweets. Save the annotated results in a file. 

4.Review at the distrbution. Is it close to even? If not, do more.

5. Take your annotated tweets - split them into train (80%) and test (20%) sets.  Process the train data and build a model (based on a TfIdf Vectorizer and an SVM). Evaluate the model on the test data sets.

6. Test your model on the remaining tweets. What does your result look like?

7. Review some of the data to identify opportunities for improvement - how might you make these models bettter?

8. Reflect on the reproducibility and the reusability of the code: what should be done to make these tools easier to apply to other datasets.



----
*ANSWER FOLLOWS - insert answer here*

1. Run some searches for tweets like 'e-cig', 'e-cigarette', 'vape' and 'vaping'. Collect a corpus of 200-300 or more tweets. You might want to save each of these result sets in files.

In [7]:
tweets1 = Tweets("e-cig", 100)
tweets1.saveTweets('part5_e-cig.json')
tweets2 = Tweets("e-cigarette", 100)
tweets2.saveTweets('part5_e-cigarette.json')
tweets3 = Tweets("vape", 100)
tweets3.saveTweets('part5_vape.json')
tweets4 = Tweets("vaping", 100)
tweets4.saveTweets('part5_vaping.json')

2. Combine these tweets into one large collection using the 'Tweet' class listed above. Save the results in a file

In [10]:
tweets1.combineTweets(tweets2)
tweets1.combineTweets(tweets3)
tweets1.combineTweets(tweets4)

In [13]:
tweets1.countTweets()
tweets1.saveTweets('part5_tweets.json')

3. Annotate 50 of these tweets as pertaining to either 'individual' or 'non-individual'. Be sure that you do at least a few of the tweets from each of the original sets. One way to do this might be to randomize the tweets. Save the annotated results in a file.

In [15]:
tweets1 = Tweets()
tweets1.readTweets('part5_tweets.json')
tweet_ids = []
for i in range(50):
    tweet_id = random.choice(list(tweets1.getIds()))
    tweet_ids.append(tweet_id)


In [16]:
i=1
for tweet_id in tweet_ids:
    print(i)
    print(tweets1.getText(tweet_id))
    i+=1

1
#TheEastLight Lee Seokcheol "members were even chocked by guitar string, and forced to smoke e-cigarette despite knowing we are minors. they even put the cigarette right to our mouth. Now we even received psychology treatment because it was traumatic" https://t.co/53msdzcbWu https://t.co/aEDiQfkoq3
2
We hope everyone is enjoying the last day of the #CWCBExpo! Don't miss the last chance to talk with our sales director Jackie(781-420-4127) at booth #507! 👈#vape #vapepen #vaporizer #vapecarts #vapeindustry #vapenation #cbd #cbdoil #cbdvape #cbdpen #cannabis #extracts #marijuana https://t.co/CIrkATzI7v
3
Dude just cruised past me riding a lawn chair taped to an electric skateboard while vaping and blasting Jack Johnson. Now I’m questioning all my life choices. https://t.co/VfFlJZKil4
4
Arizona to focus on putting out e-cigarette use among youths https://t.co/ffcO1QexWW
5
E-cigarette retail sales have surged in recent years, according to new data, and a large portion are targeting teen an

In [17]:
tweets1.addCode(tweet_ids[0], 'INDIVIDUAL')

In [18]:
tweets1.addCode(tweet_ids[1], 'NON-INDIVIDUAL')

In [19]:
tweets1.addCode(tweet_ids[2], 'INDIVIDUAL')

In [20]:
tweets1.addCode(tweet_ids[3], 'NON-INDIVIDUAL')

In [21]:
tweets1.addCode(tweet_ids[4], 'NON-INDIVIDUAL')

In [22]:
tweets1.addCode(tweet_ids[5], 'INDIVIDUAL')

In [23]:
tweets1.addCode(tweet_ids[6], 'INDIVIDUAL')

In [24]:
tweets1.addCode(tweet_ids[7], 'NON-INDIVIDUAL')

In [25]:
tweets1.addCode(tweet_ids[8], 'INDIVIDUAL')

In [26]:
tweets1.addCode(tweet_ids[9], 'NON-INDIVIDUAL')

In [27]:
tweets1.addCode(tweet_ids[10], 'NON-INDIVIDUAL')

In [28]:
tweets1.addCode(tweet_ids[11], 'NON-INDIVIDUAL')

In [29]:
tweets1.addCode(tweet_ids[12], 'NON-INDIVIDUAL')

In [30]:
tweets1.addCode(tweet_ids[13], 'INDIVIDUAL')

In [31]:
tweets1.addCode(tweet_ids[14], 'INDIVIDUAL')

In [32]:
tweets1.addCode(tweet_ids[15], 'INDIVIDUAL')

In [33]:
tweets1.addCode(tweet_ids[16], 'INDIVIDUAL')

In [34]:
tweets1.addCode(tweet_ids[17], 'INDIVIDUAL')

In [35]:
tweets1.addCode(tweet_ids[18], 'INDIVIDUAL')

In [36]:
tweets1.addCode(tweet_ids[19], 'INDIVIDUAL')

In [37]:
tweets1.addCode(tweet_ids[20], 'INDIVIDUAL')

In [38]:
tweets1.addCode(tweet_ids[21], 'INDIVIDUAL')

In [39]:
tweets1.addCode(tweet_ids[22], 'INDIVIDUAL')

In [40]:
tweets1.addCode(tweet_ids[23], 'INDIVIDUAL')

In [41]:
tweets1.addCode(tweet_ids[24], 'INDIVIDUAL')

In [42]:
tweets1.addCode(tweet_ids[25], 'INDIVIDUAL')

In [43]:
tweets1.addCode(tweet_ids[26], 'NON-INDIVIDUAL')

In [44]:
tweets1.addCode(tweet_ids[27], 'NON-INDIVIDUAL')

In [45]:
tweets1.addCode(tweet_ids[28], 'NON-INDIVIDUAL')

In [46]:
tweets1.addCode(tweet_ids[29], 'INDIVIDUAL')

In [47]:
tweets1.addCode(tweet_ids[30], 'INDIVIDUAL')

In [48]:
tweets1.addCode(tweet_ids[31], 'NON-INDIVIDUAL')

In [49]:
tweets1.addCode(tweet_ids[32], 'NON-INDIVIDUAL')

In [50]:
tweets1.addCode(tweet_ids[33], 'NON-INDIVIDUAL')

In [51]:
tweets1.addCode(tweet_ids[34], 'NON-INDIVIDUAL')

In [52]:
tweets1.addCode(tweet_ids[35], 'NON-INDIVIDUAL')

In [53]:
tweets1.addCode(tweet_ids[36], 'INDIVIDUAL')

In [54]:
tweets1.addCode(tweet_ids[37], 'INDIVIDUAL')

In [55]:
tweets1.addCode(tweet_ids[38], 'INDIVIDUAL')

In [56]:
tweets1.addCode(tweet_ids[39], 'NON-INDIVIDUAL')

In [57]:
tweets1.addCode(tweet_ids[40], 'NON-INDIVIDUAL')

In [58]:
tweets1.addCode(tweet_ids[41], 'INDIVIDUAL')

In [59]:
tweets1.addCode(tweet_ids[42], 'NON-INDIVIDUAL')

In [60]:
tweets1.addCode(tweet_ids[43], 'NON-INDIVIDUAL')

In [61]:
tweets1.addCode(tweet_ids[44], 'INDIVIDUAL')

In [62]:
tweets1.addCode(tweet_ids[45], 'NON-INDIVIDUAL')

In [63]:
tweets1.addCode(tweet_ids[46], 'NON-INDIVIDUAL')

In [64]:
tweets1.addCode(tweet_ids[47], 'INDIVIDUAL')

In [65]:
tweets1.addCode(tweet_ids[48], 'INDIVIDUAL')

In [66]:
tweets1.addCode(tweet_ids[49], 'INDIVIDUAL')

In [72]:
def combineTweetsIds(original,other):
    for otherid in other:
        tweet = tweets1.getTweet(otherid)
        searchTerm = tweets1.getSearchTerm(otherid)
        searchTime = tweets1.getSearchTime(otherid)
        original.addTweet(tweet,searchTime,searchTerm)
    

In [73]:
#tweets1.saveTweets('part5_tweets_annotated.json')
print(len(tweet_ids))
tweets = Tweets()
combineTweetsIds(tweets, tweet_ids)

50


In [77]:
tweets.countTweets()
tweets.saveTweets('part5_tweets_annotated.json')

4. Review at the distrbution. Is it close to even? If not, do more.

Close to even distrubution so continuing with the dataset.

In [78]:
tweets.getCodeProfile()

[('NON-INDIVIDUAL', 21), ('INDIVIDUAL', 28)]

In [79]:
one = two = three = four = 0
for tweet_id in tweet_ids:
    term = tweets1.getSearchTerm(tweet_id)
    if term == 'e-cig':
        one += 1
    elif term == 'e-cigarette':
        two += 1
    elif term == 'vape':
        three +=1 
    elif term == 'vaping':
        four += 1
print(one, two, three, four)

10 17 11 12


5. Take your annotated tweets - split them into train (80%) and test (20%) sets. Process the train data and build a model (based on a TfIdf Vectorizer and an SVM). Evaluate the model on the test data sets.


In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tokenizeText(text):
    nlp=getTwitterNLP()
    tokens=nlp(text)
    return filterTweetTokens(tokens)

vectorizer= TfidfVectorizer(tokenizer=tokenizeText,preprocessor=lambda x: x)
clf = LinearSVC()
pipe = Pipeline([('vectorizer', vectorizer), ('clf', clf)])

In [83]:
def flattenTweets(tweets):
    flat=[]
    for i in tweets.getIds():
        text = tweets.getText(i)
        for x in tweets.getCodes(i):
            cat = x
        pair =(text,cat)
        flat.append(pair)
    return flat

In [84]:
def getTestTrainSplit(pairs,splitFactor=0.8):
    random.shuffle(pairs)
    split=int(len(pairs)*splitFactor)
    train=pairs[:split]
    test =pairs[split:]
    return train,test

def getTestTrain(tweets,splitFactor=0.8):
    tweets = flattenTweets(tweets)
    train,test=getTestTrainSplit(tweets,splitFactor)
    return train,test

In [85]:
train,test=getTestTrain(tweets)
print(str(len(train))+ " "+str(len(test)))


39 10


In [86]:
trainTexts,trainCats=zip(*train)
testTexts,testCats=zip(*test)

In [87]:
train[0]

('#TheEastLight Lee Seokcheol "members were even chocked by guitar string, and forced to smoke e-cigarette despite knowing we are minors. they even put the cigarette right to our mouth. Now we even received psychology treatment because it was traumatic" https://t.co/53msdzcbWu https://t.co/aEDiQfkoq3',
 'INDIVIDUAL')

In [88]:
pipe.fit(trainTexts,trainCats)
preds = pipe.predict(testTexts)

6. Test your model on the remaining tweets. What does your result look like?



In [89]:
print("accuracy:", accuracy_score(testCats, preds))

accuracy: 0.5


In [90]:
def convertToNumeric(cats):
    nums =[]
    for c in cats:
        if c =='INDIVIDUAL':
            nums.append(1)
        elif c=='NON-INDIVIDUAL':
            nums.append(-1)
    return nums

In [91]:
numCats=convertToNumeric(testCats)
numPreds=convertToNumeric(preds)

In [92]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print("Precision is "+str(precision_score(numCats,numPreds,average=None)))
print("Recall is "+ str(recall_score(numCats,numPreds,average=None)))

Precision is [1.    0.375]
Recall is [0.28571429 1.        ]


7. Review some of the data to identify opportunities for improvement - how might you make these models bettter?

These models can be made better by identifying features specific to the terms being analyzed and the tweets related to those terms. Once the discriminating features are known, the models can be trained better using those features. Such features may be counting terms related to smoking, certain hashtags/emoticons always found with vaping tweets, style of tweeting by individuals, presence of URL's common in non-individual tweets, individuals using more hashtags etc.

8. Reflect on the reproducibility and the reusability of the code: what should be done to make these tools easier to apply to other datasets.

The Tweets class makes a very good attempt at making the code reproducible and reusable. The NLP routine getTwitterNLP() can be made more generalized so as to be applicable to other datasets (with few modifications if needed). Currently special cases are being hard coded, these can be recoded. Moreover, the flattenTweets() routine only considers annotation using the search term whereas above we have used the codes, the routine can be modified such that this is taken into consideration.

If datasets used are not taken from Twitter, and instead from other social media sites such as Reddit or YouTube then the code may not work very well. The routines would have to be rewritten depending on the specific characteristics of multiple datasets to make it reproducible and reusable.